In [9]:
from langchain.text_splitter import Language
from langchain.document_loaders.generic import GenericLoader
from langchain.document_loaders.parsers import LanguageParser
import os

from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.embeddings import HuggingFaceInstructEmbeddings
from langchain.chains import RetrievalQA
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from huggingface_hub import notebook_login
from transformers import pipeline
from transformers import AutoTokenizer, AutoModelForCausalLM
from langchain import HuggingFacePipeline
from langchain.text_splitter import CharacterTextSplitter
import textwrap
import sys
import torch

from langchain.embeddings import OpenAIEmbeddings

In [10]:
import APIKEY
os.environ["OPENAI_API_KEY"] = APIKEY.API_KEY_SERVICE_OPENAI

## Multiple files with code paser

In [15]:

loader = GenericLoader.from_filesystem(
        "./docs/m460bsp_clk_m460",
        glob="**/*",
        suffixes=[".h", ".c"],
        parser=LanguageParser(language=Language.C, parser_threshold=10000),
        #parser=LanguageParser(),
        show_progress=True,
    )
docs = loader.load()
len(docs)

  0%|          | 0/4 [00:00<?, ?it/s]

4

In [16]:
for document in docs:
    print(document.metadata)
#print("\n\n--8<--\n\n".join([document.page_content for document in docs]))  

{'source': 'docs\\m460bsp_clk_m460\\clk.c', 'language': <Language.C: 'c'>}
{'source': 'docs\\m460bsp_clk_m460\\clk.h', 'language': <Language.C: 'c'>}
{'source': 'docs\\m460bsp_clk_m460\\clk_reg.h', 'language': <Language.C: 'c'>}
{'source': 'docs\\m460bsp_clk_m460\\m460.h', 'language': <Language.C: 'c'>}


In [17]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
C_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.C, chunk_size=2000, chunk_overlap=200)
texts = C_splitter.split_documents(docs)
len(texts)

254

In [8]:
print(texts[0].page_content)
print(texts[0])

/**************************************************************************//**
 * @file     acmp.h
 * @version  V3.00
 * @brief    ACMP Driver Header File
 *
 * @copyright SPDX-License-Identifier: Apache-2.0
 * @copyright Copyright (C) 2020 Nuvoton Technology Corp. All rights reserved.
 ******************************************************************************/
#ifndef __ACMP_H__
#define __ACMP_H__

#ifdef __cplusplus
extern "C"
{
#endif


/** @addtogroup Standard_Driver Standard Driver
  @{
*/

/** @addtogroup ACMP_Driver ACMP Driver
  @{
*/


/** @addtogroup ACMP_EXPORTED_CONSTANTS ACMP Exported Constants
  @{
*/
page_content='/**************************************************************************//**\n * @file     acmp.h\n * @version  V3.00\n * @brief    ACMP Driver Header File\n *\n * @copyright SPDX-License-Identifier: Apache-2.0\n * @copyright Copyright (C) 2020 Nuvoton Technology Corp. All rights reserved.\n **************************************************************

In [ ]:
for document in texts:
    print(document.metadata)
#print("\n\n--8<--\n\n".join([document.page_content for document in texts]))  

## Single PDF file

In [1]:
# PyPdf + rapidocr-onnxruntime
from langchain_community.document_loaders import PyPDFLoader
loader = PyPDFLoader("./docs/en-us--TRM_M463_M467_Series_EN_Rev1.01.pdf", extract_images=True)

# UnstructuredFileLoader
#loader = UnstructuredFileLoader("./docs/TRM_M2354_Series_EN_Rev1.01.pdf")

documents = loader.load()

In [6]:
text_splitter=CharacterTextSplitter(separator='\n', chunk_size=2000, chunk_overlap=200)
text_doc=text_splitter.split_documents(documents)

In [7]:
len(text_doc)

3294

In [6]:
import tiktoken

def num_tokens_from_string(string: str, encoding_name: str) -> int:
    """Returns the number of tokens in a text string."""
    encoding = tiktoken.get_encoding(encoding_name)
    num_tokens = len(encoding.encode(string))
    return num_tokens

num_tokens = 0
for text_ele in text_doc:
    num_tokens += num_tokens_from_string("tiktoken is great!", "cl100k_base")
print(num_tokens)    

13098


In [25]:
#print(text_doc[0].page_content)
print(text_doc[1650])

page_content='M463/M467  Series  \nAug. 24, 2023  Page 1190  of 2380  Rev 1.01 \nM463/M467  SERIES TECHNICAL RE FERENCE MANUAL  UART Feature   UART0/ UART1  UART2 ~ UART9  USCI -UART  \nFIFO  16 Bytes  16 Bytes  TX: 1byte  \nRX: 2byte  \nAuto Flow Control (CTS/RTS)  √ √ √ \nIrDA √ √ - \nLIN √ - - \nRS-485 Function Mode √ √ √ \nnCTS Wake-up √ √ √ \nIncoming Data Wake-up √ √ √ \nReceived Data FIFO reached threshold  Wake-up √ √ - \nRS-485 Address Match (AAD mode) Wake -up √ √ - \nReceived Data FIFO reached threshold  Time -out \nWake-up √ √ - \nBaud Rate Compensation  √ √ - \nAuto-Baud Rate Measurement  √ √ √ \nSTOP bit Length  1, 1.5, 2 bit  1, 1.5, 2 bit  1, 2 bit  \nWord Length  5, 6, 7, 8 bits  5, 6, 7, 8 bits  6~13 bits  \nEven / Odd Parity √ √ √ \nStick Bit √ √ - \nTable 6.23-1 M463/M467 Series UART Feature s \n6.23.3  Block Diagram  \nThe UART clock control and block diagram are shown in Figure 6.23-1 and Figure 6.23-2 respectively.  \nNote: The frequency of UARTx_CLK should not b

In [7]:
loader2 = UnstructuredFileLoader("./docs/M460_SVD.txt")
documents2 = loader2.load()
text_splitter=CharacterTextSplitter(separator='\n', chunk_size=2000, chunk_overlap=200)
text_doc2=text_splitter.split_documents(documents2)
len(text_doc2)

4534

### Combine 2 different types

In [22]:
texts = texts + text_doc

In [23]:
len(texts)

6752

In [17]:
print(texts[7547].page_content)
print(texts[7547])

NuMicro ISP Flow And Command Set
Introduction:
Most of modern consumer products has the capability to upgrade its firmware code running on internal microcontroller. With this feature, the product is able to continuously support new functions after it is made and released to end customer. Nuvoton provides a ISP (In-System Programming) method to update the flash code of the NuMicro series Flash-memory-based microcontrollers.
The NuMicro ISP code is resident in LDROM (Loader ROM), it supports different I/O interfaces, including USB, UART, I2C, SPI, RS485, CAN to program or update the application code into internal APROM (Application ROM). This is a very convenient way for developer or end user to update application code of a NuMicro chip that was mounted on PCB (Printed Circuit Board).
This document describes the ISP code flow and the usage of ISP commands.
tools, and supporting http://www.nuvoton.com/
The relative datasheet, Technical Reference Manual, BSP (Board Support Package) sof

## Save the texts into FAISS in disk

In [18]:
save_path = r'D:\nu_QA_data'
#save_name = r'TRM_M2354_and_StdDriver_C_Regs_bge_s'
#save_name = r'm460_StdDriver_Regs_openai'
save_name = r'm460bsp_clk_m460_openai'

# load embedding model
print("===== Load the embedding model =====")
#embedding = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device': 'cpu'})
#embedding = HuggingFaceInstructEmbeddings(model_name='BAAI/bge-base-en-v1.5', model_kwargs={'device': 'cpu'})
#embedding = HuggingFaceInstructEmbeddings(model_name='BAAI/bge-small-en')
embedding=OpenAIEmbeddings()

# Create vectors store
print("===== Build FAISS =====")
vectorstore=FAISS.from_documents(texts, embedding)

index_path = os.path.join(save_path, save_name)
print(index_path)
vectorstore.save_local(index_path)

===== Load the embedding model =====
===== Build FAISS =====
D:\nu_QA_data\m460bsp_clk_m460_openai


## Create VB only by chosen files

In [22]:
import re
import shutil



def find_files_with_pattern(directory, patterns, dst):
    matching_path = []
    
    for root, dirs, files in os.walk(directory):
        for file in files:
            file_path = os.path.join(root, file)
            
            for pattern in patterns:
                # Check if the file name matches the pattern
                if re.search(pattern, file, re.IGNORECASE):
                    matching_path.append(file_path)
                    shutil.copy(file_path, dst)
    
    return matching_path

def find_folders_with_pattern(directory, patterns, dst):
    matching_path = []
    
    for dirname in os.listdir(directory):
        
        for pattern in patterns:
            # Check if the file name matches the pattern
            if re.search(pattern, dirname, re.IGNORECASE):
                dir_loc = os.path.join(directory, dirname)
                #print(dir_loc, dirname)
                matching_path.append(dir_loc)
                shutil.copytree(dir_loc, os.path.join(dst, dirname), dirs_exist_ok=True)
            
    return matching_path

# Example usage:
directory_path = "./m251bsp"
#file_pattern = [r"acmp", r"dac"]
file_pattern = [r"uart", r"pdma", r"crc"]
dst = r"D:\nu_QA_data\m251bsp_partial_source"

if os.path.isdir(dst):
    print("Remove the old ones!")
    #os.rmdir(dst)
    shutil.rmtree(dst) 
else:
    print("First create!")
os.mkdir(dst)    

# Find the StdDriver
matching_files = find_files_with_pattern(os.path.join(directory_path, 'StdDriver'), file_pattern, dst)

# Find the SampleCode\StdDriver
matching_files += find_folders_with_pattern(os.path.join(directory_path, 'SampleCode', 'StdDriver'), file_pattern, dst)


print(matching_files)


Remove the old ones!
['./m251bsp\\StdDriver\\inc\\crc.h', './m251bsp\\StdDriver\\inc\\pdma.h', './m251bsp\\StdDriver\\inc\\scuart.h', './m251bsp\\StdDriver\\inc\\uart.h', './m251bsp\\StdDriver\\inc\\usci_uart.h', './m251bsp\\StdDriver\\src\\crc.c', './m251bsp\\StdDriver\\src\\pdma.c', './m251bsp\\StdDriver\\src\\scuart.c', './m251bsp\\StdDriver\\src\\uart.c', './m251bsp\\StdDriver\\src\\usci_uart.c', './m251bsp\\SampleCode\\StdDriver\\CRC_CCITT', './m251bsp\\SampleCode\\StdDriver\\CRC_CRC32', './m251bsp\\SampleCode\\StdDriver\\CRC_CRC8', './m251bsp\\SampleCode\\StdDriver\\DAC_PDMA_TimerTrigger', './m251bsp\\SampleCode\\StdDriver\\EADC_PDMA_BPWM_Trigger', './m251bsp\\SampleCode\\StdDriver\\EADC_PDMA_PWM_Trigger', './m251bsp\\SampleCode\\StdDriver\\FMC_CRC32', './m251bsp\\SampleCode\\StdDriver\\I2C_PDMA_TRX', './m251bsp\\SampleCode\\StdDriver\\PDMA_BasicMode', './m251bsp\\SampleCode\\StdDriver\\PDMA_ScatterGather', './m251bsp\\SampleCode\\StdDriver\\PDMA_ScatterGather_PingPongBuffer', '.

In [23]:

loader = GenericLoader.from_filesystem(
        dst,
        glob="**/*",
        suffixes=[".h", ".c"],
        parser=LanguageParser(language=Language.C, parser_threshold=10000),
        #parser=LanguageParser(),
        show_progress=True,
    )
docs = loader.load()
len(docs)

  0%|          | 0/50 [00:00<?, ?it/s]

50

In [24]:
for document in docs:
    print(document.metadata)

{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\crc.c', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\crc.h', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\pdma.c', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\pdma.h', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\scuart.c', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\scuart.h', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\uart.c', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\uart.h', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\usci_uart.c', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\usci_uart.h', 'language': <Language.C: 'c'>}
{'source': 'D:\\nu_QA_data\\m251bsp_partial_source\\CRC_CCITT\\main.

In [25]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
C_splitter = RecursiveCharacterTextSplitter.from_language(language=Language.C, chunk_size=1000, chunk_overlap=100)
texts = C_splitter.split_documents(docs)
len(texts)

935

In [26]:
save_path = r'D:\nu_QA_data'
save_name = r'm251bsp_partial'

# load embedding model
print("===== Load the embedding model =====")
embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2',model_kwargs={'device': 'cpu'})

# Create vectors store
print("===== Build FAISS =====")
vectorstore=FAISS.from_documents(texts, embeddings)

index_path = os.path.join(save_path, save_name)
print(index_path)
vectorstore.save_local(index_path)

===== Load the embedding model =====
===== Build FAISS =====
D:\nu_QA_data\m251bsp_partial
